In [41]:
import pandas as pd
import numpy as np

from src.basic import PATH

PATH.get_target_path('financial_ts' , 'disclosure' , 20240630)


WindowsPath('d:/Coding/learndl/learndl/data/DataBase/DB_financial_ts/disclosure/2024/disclosure.20240630.feather')

In [39]:
from src.data.tushare import pro
df = pro.dividend(end_date='20240317')
df

,ts_code,end_date,ann_date,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date


In [2]:
from src.api import ModelTrainer  
app = ModelTrainer.initialize(stage = 0 , resume = 0 , checkname= 1 , override={'env.short_test':True})
app.go()

24-11-11 12:18:29|MOD:display     |: Model Specifics:
24-11-11 12:18:29|MOD:display     |: Start Process [Data] at Mon Nov 11 12:18:29 2024!


--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to lgbm_day_ShortTest!
StatusDisplay(verbosity=10) , display epoch and event information
DetailedAlphaAnalysis(use_num=avg) , record and concat each model to Alpha model instance
{'model_name': 'lgbm_day_ShortTest',
 'model_module': 'lgbm',
 'model_submodels': ['best'],
 'model_booster_type': 'lgbm',
 'model_booster_optuna': True,
 'model_booster_head': None,
 'model_data_types': ['day'],
 'model_data_labels': None,
 'random_seed': None,
 'beg_date': 20170101,
 'end_date': 20170228,
 'train_sample_method': 'sequential',
 'train_shuffle_option': 'epoch'}
{'seqlens': [{'day': 5}],
 'objective': ['mse'],
 'linear_tree': [True],
 'learning_rate': [0.3],
 'reg_lambda': [1e-05],
 'reg_alpha': [1e-07],
 'num_leaves': [31],
 'max_depth': [6],
 'min_sum_hessian_in_leaf': [1],
 'feature_fraction': [0.6],
 'bagging_fraction': [0.75],
 'force_col_wise': [True],
 'monotone_constraints': [0],
 'verbosity': 10}
try using d:

24-11-11 12:18:31|MOD:display     |: Finish Process [Data], Cost 1.8 Secs
24-11-11 12:18:31|MOD:display     |: Start Process [Fit] at Mon Nov 11 12:18:31 2024!
24-11-11 12:18:31|MOD:classes     |: First Iterance: (20170103 , 0)


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 10/10 [00:00<?, ?it/s]
[I 2024-11-11 12:18:32,805] A new study created in RDB with name: lgbm_day_ShortTest@0@20170103@best_20241111-121832_mhcvknMto9
 24%|██▎       | 9/38 [00:00<00:00, 86.77it/s]

...

,group,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
num,submodel,,,,,,,,,,,,,,,,,,,,
0,best,-3.103%,-1.958%,-1.639%,-0.546%,-1.381%,-1.147%,-0.979%,-0.770%,-0.756%,-0.822%,-0.871%,-0.560%,-0.583%,-0.593%,-0.970%,-0.605%,-0.419%,-0.211%,-0.339%,10.351%


Grouped Return Results are saved to d:\Coding\learndl\learndl\models\lgbm_day_ShortTest\detailed_analysis\group.xlsx


,factor_name,benchmark,start,end,pf,bm,excess,annualized,mdd,te,ir,calmar,turnover,mdd_period
1,best,csi300,20170104,20170302,1.94%,1.97%,-0.03%,-0.22%,0.45%,1.43%,-0.154,-0.491,3.400,20170208-20170222
2,best,csi500,20170104,20170302,1.71%,0.66%,1.05%,6.60%,0.76%,3.94%,1.676,8.712,3.400,20170125-20170208
0,best,csi1000,20170104,20170302,0.19%,-2.29%,2.48%,16.81%,0.61%,4.85%,3.463,27.584,3.400,20170125-20170208


24-11-11 12:19:46|MOD:timer       |: Main Process Finished! Cost 1 Minutes 16.1 Seconds


Analytic datas are saved to d:\Coding\learndl\learndl\models\lgbm_day_ShortTest\detailed_analysis\data.xlsx
Analytic plots are saved to d:\Coding\learndl\learndl\models\lgbm_day_ShortTest\detailed_analysis\plot.pdf


In [1]:
from src.api import ModelHiddenExtractor
ModelHiddenExtractor.update_hidden()



model_name is None, update all hidden models
{'name': 'gru_day', 'nums': None, 'submodels': ['best']}
Beware! Should be at server or short_test, but short_test is False now!
try using d:\Coding\learndl\learndl\data\DataSet/day.20240607.pt , success!
Load  2 DataBlocks...... finished! Cost 0.08 secs
Align 2 DataBlocks...... finished! Cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 316/316 [00:00<00:00, 3047.38it/s]

--------------------------------------------------------------------------------
